In [3]:
import torch
import numpy as np
from sklearn.svm import SVR, SVC
from sklearn.multioutput import MultiOutputRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [4]:
# Create sample data: input features and vector outputs
data = torch.load("/kaggle/input/das-data-v2/data.pt")
X = data["imgs"].numpy()
X = X.reshape(X.shape[0], -1)
y = data["labels"].numpy()
counts = data["counts"].numpy()
#y_norm = y / np.linalg.norm(y, axis=1, keepdims=True)

X_train, X_test, y_train, y_test, count_train, count_test = train_test_split(X, y, counts, test_size=0.2)


/tmp/ipykernel_31/2404740969.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("/kaggle/input/das-data-v2/data.pt")


In [5]:
# Create SVR model and wrap it
svr = SVR(kernel='rbf', max_iter = int(1e2))
multi_output_svr = MultiOutputRegressor(svr)
y_train_norm = y_train / np.linalg.norm(y_train, axis=1, keepdims=True)
y_test_norm = y_test / np.linalg.norm(y_test, axis=1, keepdims=True)
# Train
multi_output_svr.fit(X_train, y_train_norm)

# Predict
y_pred = multi_output_svr.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-

In [6]:
mse = np.mean((y_pred - y_test_norm)**2)
print(mse)

0.03854320029436161


In [7]:
#Create count svc
svc = SVC(kernel='rbf', max_iter = int(1e3), C = 50)
svc.fit(X_train, count_train)

# Predict
count_pred = svc.predict(X_test)

In [8]:
count_acc = np.mean(count_pred == count_test)
count_mae = np.mean(np.abs(count_pred - count_test))
print(f"Count Accuracy: {count_acc}, Count MAE: {count_mae}")
diff = count_pred - count_test
within_1 = diff <= 1
print(f"Count Off by 1:", np.mean(within_1))

Count Accuracy: 0.37992831541218636, Count MAE: 1.96415770609319
Count Off by 1: 0.8494623655913979


In [9]:
import joblib
joblib.dump(multi_output_svr, 'distribution_svr_model.pkl')
joblib.dump(svc, 'count_svc_model.pkl')

['count_svc_model.pkl']

In [10]:
distribution_pred = np.round(y_pred * count_pred[:, np.newaxis])
print(np.mean(distribution_pred == y_test))

0.5407706093189965


In [15]:
from sklearn.metrics import f1_score
f1 = 0
for i in range(distribution_pred.shape[1]):
    f1 += f1_score(distribution_pred[:, i], y_test[:, i], average = "weighted")
f1 = f1/distribution_pred.shape[1]
print(f"F1: {f1}")

F1: 0.4737655682131827
